In [2]:
pip install qiskit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Erika\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
!pip install qiskit-aer
!pip install matplotlib


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Erika\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------------- -------------------- 3.9/8.0 MB 19.6 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.0 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 13.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------------------- -------------------- 1.0/2.2 MB 8.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 4.6 MB/s eta 0:00:01
   ----------------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\Erika\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

def convert_qiskit_circuit_to_unitary(qc):
    """
    Convert a Qiskit QuantumCircuit into a unitary matrix.

    Parameters:
        qc (QuantumCircuit): Qiskit quantum circuit.

    Returns:
        np.ndarray: The unitary matrix representation of the circuit.
    """
    return Operator(qc).data  # Extract unitary matrix

# Example Usage:
# qc = QuantumCircuit(2)
# qc.h(0)
# qc.cx(0, 1)

from qiskit import QuantumCircuit

# Create a quantum circuit with 4 qubits
qc = QuantumCircuit(4)

# Layer 1
qc.cx(0, 1)
qc.cx(2, 3)

# Layer 2
qc.cx(1, 2)
qc.cx(3, 0)

# Layer 3
qc.cx(0, 2)
qc.cx(1, 3)

# Display the circuit
print(qc)


unitary_matrix = np.array(convert_qiskit_circuit_to_unitary(qc))
# print(qc)
#print("Unitary Matrix:\n", unitary_matrix)






               ┌───┐          
q_0: ──■───────┤ X ├──■───────
     ┌─┴─┐     └─┬─┘  │       
q_1: ┤ X ├──■────┼────┼────■──
     └───┘┌─┴─┐  │  ┌─┴─┐  │  
q_2: ──■──┤ X ├──┼──┤ X ├──┼──
     ┌─┴─┐└───┘  │  └───┘┌─┴─┐
q_3: ┤ X ├───────■───────┤ X ├
     └───┘               └───┘


In [1]:
import numpy as np
from qiskit.quantum_info import Pauli, Statevector

def normalize_pauli(pauli_string):
    """
    Normalize a Pauli string to match the normalized Pauli basis.
    Each single-qubit Pauli operator is divided by sqrt(2).
    """
    n = len(pauli_string)  # Number of qubits
    normalization_factor = 1 / np.sqrt(2**n)
    pauli_matrix = Pauli(pauli_string).to_matrix()
    
    return normalization_factor * pauli_matrix

def extract_qubit_pauli(pauli_string, qubits):
    """
    Extracts the relevant sub-Pauli operator for specific qubits.
    
    Parameters:
        pauli_string (str): Full Pauli string (e.g., "IZIZ").
        qubits (list): Indices of qubits to extract (e.g., [0, 1]).
    
    Returns:
        str: Sub-Pauli operator for specified qubits.
    """
    return ''.join([pauli_string[q] for q in qubits])

def calculate_gate_transition_amplitude(sd, sd_minus_1, gate, qubit_indices):
    """
    Calculate Tr(s_d U_gate s_{d-1} U_gate†) for a single 2-qubit gate.
    
    Parameters:
        sd (str): Current full Pauli operator (e.g., "IZIZ").
        sd_minus_1 (str): Previous full Pauli operator (e.g., "XXIX").
        gate (np.ndarray): Unitary matrix for a single 2-qubit gate.
        qubit_indices (list): Indices of qubits that the gate acts on.
    
    Returns:
        float: Transition amplitude for this gate.
    """
    # Extract relevant sub-Paulis
    sd_sub = extract_qubit_pauli(sd, qubit_indices)
    sd_minus_1_sub = extract_qubit_pauli(sd_minus_1, qubit_indices)
    
    # Convert to normalized matrices
    sd_matrix = Pauli(sd_sub).to_matrix() / np.sqrt(2**len(sd_sub))
    sd_minus_1_matrix = Pauli(sd_minus_1_sub).to_matrix() / np.sqrt(2**len(sd_minus_1_sub))
    
    # Compute transition amplitude
    transformed_sd_minus_1 = gate @ sd_minus_1_matrix @ gate.conj().T
    return np.trace(sd_matrix @ transformed_sd_minus_1).real

def calculate_non_gate_transition_amplitude(sd, sd_minus_1, qubit_indices):
    """
    Calculate transition amplitude for qubits not acted upon by any gate.
    
    Parameters:
        sd (str): Current full Pauli operator.
        sd_minus_1 (str): Previous full Pauli operator.
        qubit_indices (list): Indices of qubits not acted upon by gates.
    
    Returns:
        float: Transition amplitude for non-gate qubits.
    """
    if not qubit_indices:  # If empty, return 1.0
        return 1.0
        
    # Extract relevant sub-Paulis
    sd_sub = extract_qubit_pauli(sd, qubit_indices)
    sd_minus_1_sub = extract_qubit_pauli(sd_minus_1, qubit_indices)
    
    # Convert to normalized matrices
    sd_matrix = Pauli(sd_sub).to_matrix() / np.sqrt(2**len(sd_sub))
    sd_minus_1_matrix = Pauli(sd_minus_1_sub).to_matrix() / np.sqrt(2**len(sd_minus_1_sub))
    
    # For non-gate qubits, the transition is just Tr(sd_sub · sd_minus_1_sub)
    return np.trace(sd_matrix @ sd_minus_1_matrix).real

def calculate_layer_transition_amplitude(sd, sd_minus_1, layer_gates, n_qubits):
    """
    Calculate transition amplitude for an entire layer of gates.
    
    Parameters:
        sd (str): Current full Pauli operator (e.g., "IZIZ").
        sd_minus_1 (str): Previous full Pauli operator (e.g., "XXIX").
        layer_gates (list of tuples): List of gates in the layer. Each tuple contains:
            - The unitary matrix for a 2-qubit gate.
            - The indices of qubits that the gate acts on.
        n_qubits (int): Total number of qubits in the system.
    
    Returns:
        float: Total transition amplitude for this layer.
    """
    layer_amplitude = 1.0
    
    # Track which qubits are acted upon by gates
    acted_qubits = set()
    for _, qubit_indices in layer_gates:
        acted_qubits.update(qubit_indices)
    
    # Calculate transition amplitude for each gate
    for gate, qubit_indices in layer_gates:
        gate_amplitude = calculate_gate_transition_amplitude(sd, sd_minus_1, gate, qubit_indices)
        layer_amplitude *= gate_amplitude
    
    # Calculate transition amplitude for qubits not acted upon by any gate
    non_gate_qubits = [i for i in range(n_qubits) if i not in acted_qubits]
    if non_gate_qubits:
        non_gate_amplitude = calculate_non_gate_transition_amplitude(sd, sd_minus_1, non_gate_qubits)
        layer_amplitude *= non_gate_amplitude
    
    return layer_amplitude

def calculate_input_overlap(s0):
    """
    Calculate Tr(s0 |0^n><0^n|).
    
    Parameters:
        s0 (str): Initial Pauli operator (e.g., "IZIZ").
    
    Returns:
        float: Input overlap.
    """
    # Check if s0 contains only I and Z (legal condition)
    if not all(op in ['I', 'Z'] for op in s0):
        return 0.0
        
    n = len(s0)  # Number of qubits
    
    # For a legal initial Pauli operator with only I and Z:
    # Count Z operators to determine sign
    z_count = s0.count('Z')
    
    # Normalization factor
    norm_factor = 1.0 / np.sqrt(2**n)
    
    # Return normalized overlap with sign based on Z count
    return norm_factor * ((-1)**z_count)

def calculate_output_overlap(x, sd):
    """
    Calculate Tr(|x><x| s_d).
    
    Parameters:
        x (str): Output state as a binary string (e.g., "0000").
        sd (str): Final Pauli operator (e.g., "ZZII").
    
    Returns:
        float: Output overlap.
    """
    # Check if sd contains only I and Z (legal condition)
    if not all(op in ['I', 'Z'] for op in sd):
        return 0.0
    
    n = len(sd)  # Number of qubits
    
    # Normalization factor
    norm_factor = 1.0 / np.sqrt(2**n)
    
    # Calculate sign based on Z operators and corresponding bits in x
    sign = 1
    for i, op in enumerate(sd):
        if op == 'Z' and x[i] == '1':
            sign *= -1
            
    return norm_factor * sign

def compute_fourier_coefficient(C, s, x):
    """
    Compute f(C, s, x) for a given circuit C and Pauli path s.
    
    Parameters:
        C (list of list of tuples): Circuit as a list of layers,
                                    where each layer contains tuples of gates and their acting qubits.
                                    Example: [[(CNOT01, [0, 1]), (CNOT23, [2, 3])], ...]
        s (list of str): Pauli path as a list of strings (e.g., ["IZIZ", "YIXI", "XXIX", "ZZII"]).
        x (str): Output state as a binary string (e.g., "0000").
    
    Returns:
        float: Fourier coefficient f(C, s, x).
    """
    n = len(s[0])  # Number of qubits
    d = len(C)     # Depth of the circuit
    
    # Check if path is legal (s0 and sd contain only I and Z)
    if not all(op in ['I', 'Z'] for op in s[0]) or not all(op in ['I', 'Z'] for op in s[-1]):
        return 0.0
    
    # Input overlap
    input_overlap = calculate_input_overlap(s[0])
    if input_overlap == 0:
        return 0.0
    
    # Transition amplitudes
    transition_amplitude = 1.0
    for i in range(d):
        layer_amplitude = calculate_layer_transition_amplitude(s[i+1], s[i], C[i], n)
        transition_amplitude *= layer_amplitude
        if transition_amplitude == 0:
            return 0.0
    
    # Output overlap
    output_overlap = calculate_output_overlap(x, s[-1])
    
    return input_overlap * transition_amplitude * output_overlap

# Example usage
if __name__ == "__main__":
    # Define gates
    CNOT = np.array([[1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 0, 1],
                      [0, 0, 1, 0]])
    
    CZ = np.array([[1, 0, 0, 0],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, -1]])
    
    SWAP = np.array([[1, 0, 0, 0],
                      [0, 0, 1, 0],
                      [0, 1, 0, 0],
                      [0, 0, 0, 1]])
    
    # Define circuit
    circuit = [
        [(CNOT, [0, 1]), (CZ, [2, 3])],    # Layer 1
        [(SWAP, [0, 1])],                  # Layer 2
        [(CNOT, [2, 3])]                   # Layer 3
    ]
    
    # Define Pauli path
    pauli_path = ["IZIZ", "YIIX", "ZZIX", "ZZIZ"]

    
    # Define output state
    output_state = "0000"
    
    # Calculate Fourier coefficient
    result = compute_fourier_coefficient(circuit, pauli_path, output_state)
    print(f"Fourier coefficient f(C, s, x) = {result}")

Fourier coefficient f(C, s, x) = 0.0


In [4]:
# Define gates
CZ = np.array([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1, 0],
               [0, 0, 0, -1]])

# Define circuit with CZ gates
circuit = [
    [(CZ, [0, 1]), (CZ, [2, 3])],  # Layer 1
    [(CZ, [0, 1]), (CZ, [2, 3])],  # Layer 2
    [(CZ, [0, 1]), (CZ, [2, 3])]   # Layer 3
]

# Define Pauli path
pauli_path = ["ZZZZ", "ZZZZ", "ZZZZ", "ZZZZ"]

# Define output state
output_state = "0000"

# Compute Fourier coefficient
result = compute_fourier_coefficient(circuit, pauli_path, output_state)
print(f"Fourier coefficient f(C, s, x) = {result}")  


Fourier coefficient f(C, s, x) = 0.0625


In [5]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit import assemble
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Pauli
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Create a 2-qubit quantum circuit
qc = QuantumCircuit(2)

# Step 2: Apply 2-qubit Pauli operators
qc.append(Pauli("XZ").to_instruction(), [0, 1])  # X on qubit 0, Z on qubit 1
qc.append(Pauli("YY").to_instruction(), [0, 1])  # Y on qubit 0, Y on qubit 1
qc.append(Pauli("ZX").to_instruction(), [0, 1])  # Z on qubit 0, X on qubit 1

# Step 3: Measure qubits
qc.measure_all()

# Step 4: Draw the circuit
print("Quantum Circuit:")
print(qc.draw())

# Step 5: Simulate using AerSimulator
simulator = Aer.get_backend('aer_simulator')
result = simulator.run(qc, shots=1024).result()  # Run the transpiled circuit


# Step 6: Get measurement results
counts = result.get_counts(qc)

# Step 7: Plot histogram of results
print("Measurement Results:", counts)
plot_histogram(counts)
plt.show()

Quantum Circuit:
        ┌────────────┐┌────────────┐┌────────────┐ ░ ┌─┐   
   q_0: ┤0           ├┤0           ├┤0           ├─░─┤M├───
        │  Pauli(XZ) ││  Pauli(YY) ││  Pauli(ZX) │ ░ └╥┘┌─┐
   q_1: ┤1           ├┤1           ├┤1           ├─░──╫─┤M├
        └────────────┘└────────────┘└────────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════════════════════════╩══╩═
                                                      0  1 
Measurement Results: {'00': 1024}
